In [2]:
import re
import numpy as np
import joblib
from sentence_transformers import SentenceTransformer
from sklearn.svm import LinearSVC

def preprocess_text(text: str) -> str:
    """
    Минимальная предобработка текста:
    - Удаляет переносы строк, табуляции, начальные спецсимволы
    - Нормализует пробелы
    - Сохраняет всё важное: пунктуацию, эмодзи, хештеги, регистр
    """
    if not isinstance(text, str):
        text = str(text)
    # Удаляем \n, \t и другие control-символы (кроме пробела)
    text = re.sub(r"[\n\t\r]+", " ", text)
    # Убираем множественные пробелы
    text = re.sub(r"\s+", " ", text)
    # Убираем пробелы по краям
    return text.strip()

class SentimentClassifier:
    def __init__(self, embedder='paraphrase-multilingual-MiniLM-L12-v2', model_path=None):
        self.embedding_model = SentenceTransformer(embedder)
        self.classifier = None
        if model_path:
            self.load_model(model_path)
    
    def _preprocess(self, texts):
        """Применяет preprocess_text к одному или списку текстов."""
        if isinstance(texts, str):
            return preprocess_text(texts)
        return [preprocess_text(t) for t in texts]
    
    def fit(self, texts, labels):
        print("Предобработка текстов...")
        texts_clean = self._preprocess(texts)
        print("Векторизация...")
        embeddings = self.embedding_model.encode(texts_clean, show_progress_bar=True)
        print("Обучение LinearSVC...")
        self.classifier = LinearSVC(
            C=0.1,
            penalty='l2',
            loss='squared_hinge',
            max_iter=2000,
            random_state=42,
            class_weight=None
        )
        self.classifier.fit(embeddings, labels)
        print("Обучение завершено.")
    
    def predict(self, texts):
        if self.classifier is None:
            raise ValueError("Модель не обучена и не загружена!")
        
        single_input = isinstance(texts, str)
        texts_clean = self._preprocess(texts)
        embeddings = self.embedding_model.encode([texts_clean], show_progress_bar=False)
        predictions = self.classifier.predict(embeddings)
        
        # Преобразуем числовые метки в строки, если нужно
        if len(predictions) > 0 and isinstance(predictions[0], (int, np.integer)):
            label_map = {0: "negative", 1: "positive"}
            predictions = [label_map.get(p, p) for p in predictions]
        
        return predictions[0] if single_input else predictions
    
    def predict_proba(self, texts):
        if self.classifier is None:
            raise ValueError("Модель не обучена и не загружена!")
        
        single_input = isinstance(texts, str)
        texts_clean = self._preprocess(texts)
        embeddings = self.embedding_model.encode([texts_clean], show_progress_bar=False)
        scores = self.classifier.decision_function(embeddings)
        proba = 1 / (1 + np.exp(-scores))
        return float(proba[0]) if single_input else proba.tolist()
    
    def save_model(self, path):
        if self.classifier is None:
            raise ValueError("Нечего сохранять!")
        joblib.dump(self.classifier, path)
        print(f"Модель сохранена в {path}")
    
    def load_model(self, path):
        self.classifier = joblib.load(path)
        # print(f"Модель загружена из {path}")

In [4]:
# ==============================
# Пример использования
# ==============================

if __name__ == "__main__":
    # --- Этап 1: Обучение (делается один раз) ---
    # texts = ["I love this!", "Этот товар ужасный", ...]
    # labels = ["positive", "negative", ...]
    # clf = SentimentClassifier()
    # clf.fit(texts, labels)
    # clf.save_model("sentiment_model.joblib")
    
    # --- Этап 2: Инференс (в production) ---
    clf = SentimentClassifier(model_path="../models/LinearSVC.joblib")
    
    # Одиночный текст
    print("This is amazing!", clf.predict("This is amazing!"))  # → positive
    print("Ужасный сервис", clf.predict("Ужасный сервис"))    # → negative
    print(f" 'Great job!' - Уверенность: {clf.predict_proba('Great job!'):.2f}")
    
    # Батч
    texts = [
        "I hate this product",
        "Отлично сработано!",
        "Not bad, but could be better"
    ]
    predictions = clf.predict(texts)
    confidences = clf.predict_proba(texts)
    
    for text, pred, conf in zip(texts, predictions, confidences):
        print(f"[{pred.upper()} ({conf:.2f})] {text}")

This is amazing! positive
Ужасный сервис negative
 'Great job!' - Уверенность: 0.96
[NEGATIVE (0.15)] I hate this product


## Chat bot like

In [6]:
clf = SentimentClassifier(model_path="../models/LinearSVC.joblib")

if __name__ == "__main__":
    # Инференс (в production) ---    

    user_input = input("Ваш текст (для завершения введите 'exit' или 'выход'): ")
    
    while user_input!='exit' and user_input!='выход':
        print('Пользователь:')
        print('\t', user_input, '\n')

        print('Ответ классификатора:')
        print('\t', clf.predict(user_input), '\n')
        print(f"Уверенность: {clf.predict_proba(user_input):.2f}")
        print('----------')
        
        user_input = input("Ваш текст: ")

Ваш текст (для завершения введите 'exit' или 'выход'):  да это клево чувак


Пользователь:
	 да это клево чувак 

Ответ классификатора:
	 negative 

Уверенность: 0.39
----------


Ваш текст:  не понимаю


Пользователь:
	 не понимаю 

Ответ классификатора:
	 negative 

Уверенность: 0.22
----------


Ваш текст:  понимаю


Пользователь:
	 понимаю 

Ответ классификатора:
	 positive 

Уверенность: 0.71
----------


Ваш текст:  выход
